# Sentinel 2 Collection

In [1]:
import ee
ee.Initialize()

In [2]:
from geetools import collection, tools, ui

In [3]:
from ipygee import *

This means that the default collection for Landsat 1 is RAW and MSS, for Landsat 5 is SR and TM

# HELPERS

In [4]:
def print_values(image):
    print(tools.image.getValue(image, image.geometry().centroid(), 30, side='client'))

## Sentinel 2 SR

In [5]:
s2sr = collection.Sentinel2SR()

In [6]:
collection.info(s2sr)

{'algorithms': {'hollstein': <function geetools.cloud_mask.applyHollstein(image, options=('cloud', 'snow', 'shadow', 'water', 'cirrus'), aerosol='B1', blue='B2', green='B3', red_edge1='B5', red_edge2='B6', red_edge3='B7', red_edge4='B8A', water_vapor='B9', cirrus='B10', swir='B11')>,
  'scl_masks': <bound method Sentinel2.SclMasks of <geetools.collection.sentinel.Sentinel2 object at 0x7f3764289908>>},
 'band_names': ['aerosol',
  'blue',
  'green',
  'red',
  'red_edge_1',
  'red_edge_2',
  'red_edge_3',
  'nir',
  'red_edge_4',
  'water_vapor',
  'swir',
  'swir2',
  'aerosol_thickness',
  'water_vapor_pressure',
  'scene_classification_map',
  'qa10',
  'qa20',
  'qa60'],
 'bands': ['B1',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'B8',
  'B8A',
  'B9',
  'B11',
  'B12',
  'AOT',
  'WVP',
  'SCL',
  'QA10',
  'QA20',
  'QA60'],
 'bit_bands': ['QA10', 'QA20', 'QA60'],
 'classification_bands': ['SCL'],
 'cloud_cover': 'CLOUD_COVERAGE_ASSESSMENT',
 'ee_collection': <ee.imagecolle

In [7]:
s2sr_i = ee.Image(s2sr.collection.first())

In [8]:
for band in s2sr.bands:
    print(band.id, band.name)

B1 aerosol
B2 blue
B3 green
B4 red
B5 red_edge_1
B6 red_edge_2
B7 red_edge_3
B8 nir
B8A red_edge_4
B9 water_vapor
B11 swir
B12 swir2
AOT aerosol_thickness
WVP water_vapor_pressure
SCL scene_classification_map
QA10 qa10
QA20 qa20
QA60 qa60


## Rename

In [9]:
s2sr_i = s2sr_i.addBands(ee.Image.constant(0).rename('test'))

In [10]:
s2sr_renamed = s2sr.rename(s2sr_i)

In [11]:
eprint(s2sr_renamed.bandNames())

## Rename collection

In [12]:
s2col = s2sr.collection

In [13]:
s2col_renamed = s2col.map(lambda img: s2sr.rename(img))

In [15]:
tools.imagecollection.getValues(s2col_renamed.limit(2), ee.Image(s2col.first()).geometry().centroid(), 20, side='client')

{'20180326T084601_20180326T085428_T35RLP': {'MSK_CLDPRB': 0.0,
  'MSK_SNWPRB': 0.0,
  'TCI_B': 206.0,
  'TCI_G': 255.0,
  'TCI_R': 255.0,
  'aerosol': 1674.0,
  'aerosol_thickness': 198.0,
  'blue': 2020.0,
  'green': 2866.0,
  'nir': 5023.0,
  'qa10': 0.0,
  'qa20': 0.0,
  'qa60': 0.0,
  'red': 3926.0,
  'red_edge_1': 4244.0,
  'red_edge_2': 4408.0,
  'red_edge_3': 4633.0,
  'red_edge_4': 4913.0,
  'scene_classification_map': 5.0,
  'swir': 6598.0,
  'swir2': 5376.0,
  'water_vapor': 4999.0,
  'water_vapor_pressure': 446.0},
 '20180326T084601_20180326T085428_T35RLQ': {'MSK_CLDPRB': None,
  'MSK_SNWPRB': None,
  'TCI_B': None,
  'TCI_G': None,
  'TCI_R': None,
  'aerosol': None,
  'aerosol_thickness': None,
  'blue': None,
  'green': None,
  'nir': None,
  'qa10': None,
  'qa20': None,
  'qa60': None,
  'red': None,
  'red_edge_1': None,
  'red_edge_2': None,
  'red_edge_3': None,
  'red_edge_4': None,
  'scene_classification_map': None,
  'swir': None,
  'swir2': None,
  'water_vapor'

## SCL Mask

In [16]:
mask = s2sr.SclMasks(s2sr_i)

## Visualization

In [17]:
Map = Map()

In [18]:
Map.show()

Map(basemap={'max_zoom': 19, 'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'attribution': 'Map …

### renamed image

In [17]:
p = ee.Geometry.Point([-71, -41])

In [18]:
s2col_renamed = s2col_renamed.filterBounds(p)

In [19]:
s2sr_renamed = s2col_renamed.first()

In [22]:
vis = s2sr.visualization('NSR', True)

In [23]:
Map.addLayer(s2sr_renamed, vis, 'S2 SR Renamed NSR')

In [24]:
Map.centerObject(s2sr_renamed)

## SCL

In [25]:
visSCL = s2sr.visualization('SCL', True)

In [25]:
Map.addLayer(s2sr_renamed, visSCL, 'SCL')

## Masks

In [17]:
s2sr.bitOptions(True)

{'qa60': ['cloud', 'cirrus']}

In [18]:
masked = s2sr.applyMask(s2sr_renamed, 'qa60', renamed=True)

In [29]:
Map.addLayer(masked, vis, 'Masked')

## Hollstein

In [29]:
s2toa = collection.Sentinel2TOA()

In [30]:
i = ee.Image(s2toa.collection.filterBounds(p).filterMetadata(s2toa.cloud_cover, 'less_than', 50).first())

In [31]:
Map.addLayer(i, s2toa.visualization('NSR'), 'TOA')

In [32]:
holl = s2toa.algorithms['hollstein']

In [33]:
help(holl)

Help on function applyHollstein in module geetools.cloud_mask:

applyHollstein(image, options=('cloud', 'snow', 'shadow', 'water', 'cirrus'), aerosol='B1', blue='B2', green='B3', red_edge1='B5', red_edge2='B6', red_edge3='B7', red_edge4='B8A', water_vapor='B9', cirrus='B10', swir='B11')
    Apply Hollstein mask



In [34]:
iholl = holl(i, ['cloud', 'shadow', 'snow'])

In [35]:
Map.addLayer(iholl, s2toa.visualization('NSR'), 'Masked TOA')